# Import Libraries

In [1]:
import random
import numpy as np
import torch as th
from torch import nn
import gym
import minerl
from tqdm.notebook import tqdm
from colabgymrender.recorder import Recorder
#from pyvirtualdisplay import Display
from sklearn.cluster import KMeans
import logging
logging.disable(logging.ERROR) # reduce clutter, remove if something doesn't work to see the error logs.

/home/olli/gits/Minecraft72/venv/lib/python3.9/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [2]:
from deploy_baseline import ResearchPotatoWrapper
from tqdm import trange
import json

testdict = {'reward':[12,34,0],'steps':[2000,2000,34]}
#json.dump(testdict,open('test.json',"w"))
#json.load(open('test.json','r'))

In [3]:
from os import listdir,makedirs
from os.path import join
from collections import defaultdict
modeldir = 'research_baseline_models'
modeldirs = [join(modeldir,d) for d in listdir(modeldir)]

In [4]:
number_of_episodes = 2
episode_len = 100

In [5]:
def eval(modeldir):
    modelname = modeldir.split('/')[1]
    env = gym.make('MineRLTreechopVectorObf-v0')


    potato = ResearchPotatoWrapper(modeldir)
    
    
    episode_rewards = defaultdict(lambda: [])
    
    for episode in trange(number_of_episodes):
        env = Recorder(env, join('./video',modelname,str(episode)), fps=24)
        obs = env.reset()
        done = False
        total_reward = 0
        steps = 0

        while not done:
     
            minerl_action = potato.predict_action(obs)

            obs, reward, done, info = env.step(minerl_action)
            total_reward += reward
            steps += 1
            if steps >= episode_len:
                break

        env.release()
        #env.play()
        print(f'Episode #{episode + 1} reward: {total_reward}\t\t episode length: {steps}\n')
        episode_rewards['reward'].append(total_reward)
        episode_rewards['steps'].append(steps)
        
    env.close()
    makedirs(join('rewards'),exist_ok=True)
    print("Saving rewards")
    json.dump(episode_rewards,open(join('rewards','modelname_rewards.json'),'w'))
    return episode_rewards

In [ ]:
for modeldir in modeldirs:
    rewards = []
    rewards.append(eval(modeldir))
    